<a href="https://colab.research.google.com/github/minji-sung/minji-sung/blob/main/1_%E1%84%8B%E1%85%AA%E1%86%AB_%E1%84%89%E1%85%B5%E1%84%80%E1%85%A8%E1%84%8B%E1%85%A7%E1%86%AF_%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5_%E1%84%88%E1%85%A9%E1%86%B8%E1%84%80%E1%85%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. importing packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
!pip install pmdarima

In [ ]:
#Standard Imports
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib.pylab import rcParams
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults,ARMA,ARMAResults
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

## 2. gathering data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/bigcon

/content/drive/MyDrive/Colab Notebooks/bigcon


In [ ]:
df = pd.read_csv('./new_data0901.csv')

In [ ]:
df['date'] = pd.to_datetime(df['date'])

## 3. 범주형 데이터: 흰다리새우, 오징어, 연어

In [ ]:
df[df.name == '흰다리새우']

,date,type,ctry1,ctry2,purpose,cat1,cat2,name,import_type,price,year,소비자물가 총지수(2015=100),소비자물가상승률(%),식료품 · 비주류음료,주류 및 담배,의류 및 신발,주택 ·수도 · 전기 및 연료,가정용품 및 가사 서비스,보건,교통,통신,오락 및 문화,교육,음식 및 숙박,기타 상품 및 서비스,month,weekofyear,환율(달러),WTi,Brent,Dubai
14,2016-01-04,수산물,중국,중국,판매용,갑각류,새우,흰다리새우,"냉동,살,자숙",8.204321,2016,101.0,1.0,2.3,0.7,1.8,-0.8,1.6,1.0,-2.2,0.1,1.8,1.6,2.5,3.4,1,1,1187.520,34.626,35.034,30.4060
24,2016-01-04,수산물,인도,인도,판매용,갑각류,새우,흰다리새우,냉동,9.059677,2016,101.0,1.0,2.3,0.7,1.8,-0.8,1.6,1.0,-2.2,0.1,1.8,1.6,2.5,3.4,1,1,1187.520,34.626,35.034,30.4060
33,2016-01-04,수산물,말레이시아,말레이시아,판매용,갑각류,새우,흰다리새우,냉동,7.189045,2016,101.0,1.0,2.3,0.7,1.8,-0.8,1.6,1.0,-2.2,0.1,1.8,1.6,2.5,3.4,1,1,1187.520,34.626,35.034,30.4060
52,2016-01-04,수산물,사우디아라비아,사우디아라비아,판매용,갑각류,새우,흰다리새우,냉동,5.700000,2016,101.0,1.0,2.3,0.7,1.8,-0.8,1.6,1.0,-2.2,0.1,1.8,1.6,2.5,3.4,1,1,1187.520,34.626,35.034,30.4060
76,2016-01-04,수산물,베트남,베트남,판매용,갑각류,새우,흰다리새우,"냉동,살,포장횟감",10.863794,2016,101.0,1.0,2.3,0.7,1.8,-0.8,1.6,1.0,-2.2,0.1,1.8,1.6,2.5,3.4,1,1,1187.520,34.626,35.034,30.4060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51259,2020-12-28,수산물,베트남,베트남,판매용,갑각류,새우,흰다리새우,"냉동,살,자숙,포장횟감",16.770000,2020,105.4,0.5,4.4,0.3,0.7,0.5,0.0,1.5,-1.8,-2.1,-1.0,-2.1,1.0,2.0,12,53,1095.775,48.135,51.280,51.0575
51276,2020-12-28,수산물,태국,태국,판매용,갑각류,새우,흰다리새우,"냉동,살,자숙,포장횟감",21.040000,2020,105.4,0.5,4.4,0.3,0.7,0.5,0.0,1.5,-1.8,-2.1,-1.0,-2.1,1.0,2.0,12,53,1095.775,48.135,51.280,51.0575
51281,2020-12-28,수산물,페루,페루,판매용,갑각류,새우,흰다리새우,냉동,6.140000,2020,105.4,0.5,4.4,0.3,0.7,0.5,0.0,1.5,-1.8,-2.1,-1.0,-2.1,1.0,2.0,12,53,1095.775,48.135,51.280,51.0575
51284,2020-12-28,수산물,태국,태국,판매용,갑각류,새우,흰다리새우,"냉동,살",9.620000,2020,105.4,0.5,4.4,0.3,0.7,0.5,0.0,1.5,-1.8,-2.1,-1.0,-2.1,1.0,2.0,12,53,1095.775,48.135,51.280,51.0575


In [ ]:
df = df[['date','cat2', 'name', 'price']]

In [ ]:
df_salmon = df[df.cat2 == '연어']
df_shrimp = df[df.name == '흰다리새우']
df_cala = df[df.cat2 == '오징어']

In [ ]:
df_salmon = df_salmon.groupby(by = 'date').mean() 
df_shrimp = df_shrimp.groupby(by = 'date').mean() 
df_cala = df_cala.groupby(by = 'date').mean() 

In [ ]:
df_salmon.to_csv('./working directory2/salmon.csv')
df_shrimp.to_csv('./working directory2/shrimp.csv')
df_cala.to_csv('./working directory2/cala.csv')

## 4-1. 세분화된 데이터: 연어

In [ ]:
df = pd.read_csv('./new_data0901.csv')

In [ ]:
df_salmon = df[df.cat2 == '연어']

In [ ]:
df = df[['date',	'ctry1',	'ctry2',	'purpose',	'cat2',	'import_type',	'price',	'year']]

In [ ]:
df_salmon = df_salmon[df_salmon.ctry1 == '노르웨이']
df_salmon

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
35,2016-01-04,노르웨이,노르웨이,자사제품제조용,연어,냉장,8.966241,2016
125,2016-01-04,노르웨이,노르웨이,판매용,연어,"냉장,필렛(F)",12.736953,2016
141,2016-01-04,노르웨이,노르웨이,판매용,연어,"냉장,포장횟감,필렛(F)",18.577778,2016
142,2016-01-04,노르웨이,노르웨이,판매용,연어,냉동,6.287863,2016
168,2016-01-04,노르웨이,노르웨이,판매용,연어,"냉장,필렛(F),횟감",13.562398,2016
...,...,...,...,...,...,...,...,...
51296,2020-12-28,노르웨이,노르웨이,자사제품제조용,연어,냉장,8.750000,2020
51309,2020-12-28,노르웨이,노르웨이,판매용,연어,"냉장,포장횟감,필렛(F)",29.740000,2020
51320,2020-12-28,노르웨이,노르웨이,판매용,연어,"냉장,필렛(F)",15.290000,2020
51332,2020-12-28,노르웨이,노르웨이,자사제품제조용,연어,"냉장,필렛(F)",14.260000,2020


4 가지 수입형태: ['냉장,필렛(F)', '냉장,포장횟감,필렛(F)', '냉장,필렛(F),횟감', '냉장']

In [ ]:
df_salmon_1 = df_salmon[df_salmon.import_type == '냉장,필렛(F)']
df_salmon_2 = df_salmon[df_salmon.import_type == '냉장,포장횟감,필렛(F)']
df_salmon_3 = df_salmon[df_salmon.import_type == '냉장,필렛(F),횟감']
df_salmon_4 = df_salmon[df_salmon.import_type == '냉장']


In [ ]:
# salmon_1 은 판매용과 자사제품제조용 분리

df_salmon_1_판매용 = df_salmon_1[df_salmon_1.purpose == '판매용']
df_salmon_1_자사 = df_salmon_1[df_salmon_1.purpose == '자사제품제조용']


In [ ]:
# salmon_2 는 판매용만

df_salmon_2 = df_salmon_2[df_salmon_2.purpose == '판매용']

In [ ]:
# salmon_3 도 판매용만
df_salmon_3 = df_salmon_3[df_salmon_3.purpose == '판매용']

In [ ]:
# salmon_4 는 판매용과 자사제조
df_salmon_4_판매용 = df_salmon_4[df_salmon_4.purpose == '판매용']
df_salmon_4_자사 = df_salmon_4[df_salmon_4.purpose == '자사제품제조용']


In [ ]:
def cleaning(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe = dataframe[['date', 'price']]
    dataframe.index = dataframe['date']
    dataframe.set_index('date', inplace = True)
    return dataframe 

In [ ]:
df_salmon_1_자사 = cleaning(df_salmon_1_자사)
df_salmon_1_판매용 = cleaning(df_salmon_1_판매용)
df_salmon_2 = cleaning(df_salmon_2)
df_salmon_3 = cleaning(df_salmon_3)
df_salmon_4_자사 = cleaning(df_salmon_4_자사)
df_salmon_4_판매용 = cleaning(df_salmon_4_판매용)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# our salmon data 
# df_salmon_1_자사 ('냉장,필렛(F)')
# df_salmon_1_판매용 ('냉장,필렛(F)')
# df_salmon_2 ('냉장,포장횟감,필렛(F)')
# df_salmon_3 ('냉장,필렛(F),횟감')
# df_salmon_4_자사 ('냉장')
# df_salmon_4_판매용 ('냉장')

df_salmon_1_자사.to_csv('./working directory/연어_노르웨이_냉장필렛_자사제품제조용.csv')
df_salmon_1_판매용.to_csv('./working directory/연어_노르웨이_냉장필렛_판매용.csv')
df_salmon_2.to_csv('./working directory/연어_노르웨이_냉장포장횟감필렛.csv')
df_salmon_3.to_csv('./working directory/연어_노르웨이_냉장필렛횟감.csv')
df_salmon_4_자사.to_csv('./working directory/연어_노르웨이_냉장_자사제품제조용.csv')
df_salmon_4_판매용.to_csv('./working directory/연어_노르웨이_냉장_판매용.csv')


## 4-2. 세분화된 데이터: 오징어

In [ ]:
df_cala = df[df.cat2 == '오징어']

In [ ]:
df_cala
# 페루, 중국, 베트남, 칠레 
# ['냉동,동체,자숙', '냉동,다리', '냉동,지느러미']

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
5,2016-01-04,칠레,칠레,판매용,오징어,"냉동,다리",0.945644,2016
37,2016-01-04,페루,페루,판매용,오징어,"냉동,지느러미",0.600000,2016
58,2016-01-04,태국,태국,판매용,오징어,"냉동,슬라이스(S),포장횟감",13.978644,2016
82,2016-01-04,베트남,베트남,판매용,오징어,"냉동,슬라이스(S),포장횟감",11.811334,2016
101,2016-01-04,페루,페루,자사제품제조용,오징어,"냉동,동체,자숙",3.716541,2016
...,...,...,...,...,...,...,...,...
51282,2020-12-28,페루,페루,판매용,오징어,"냉동,다리",2.160000,2020
51283,2020-12-28,페루,페루,판매용,오징어,"냉동,동체,자숙",2.910000,2020
51289,2020-12-28,페루,페루,판매용,오징어,"냉동,지느러미",1.330000,2020
51294,2020-12-28,중국,중국,판매용,오징어,"냉동,동체",2.090000,2020


### 4-2-1. 오징어페루

In [ ]:
df_cala_peru = df_cala[df_cala.ctry1 == '페루']

In [ ]:
df_c_p = df_cala_peru

In [ ]:
df_c_p_1 = df_c_p[df_c_p.import_type == '냉동,동체,자숙']
df_c_p_2 = df_c_p[df_c_p.import_type == '냉동,다리']
df_c_p_3 = df_c_p[df_c_p.import_type == '냉동,지느러미']


In [ ]:
df_c_p_1_자사 = df_c_p_1[df_c_p_1.purpose == '자사제품제조용']
df_c_p_1_판매 = df_c_p_1[df_c_p_1.purpose == '판매용']


In [ ]:
df_c_p_1_자사 = cleaning(df_c_p_1_자사)
df_c_p_1_판매 = cleaning(df_c_p_1_판매)
df_c_p_2 = cleaning(df_c_p_2)
df_c_p_3 = cleaning(df_c_p_3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_c_p_1_자사.to_csv('./working directory/오징어_페루_냉동동체자숙_자사제품제조용.csv')
df_c_p_1_판매.to_csv('./working directory/오징어_페루_냉동동체자숙_판매용.csv')
df_c_p_2.to_csv('./working directory/오징어_페루_냉동다리.csv')
df_c_p_3.to_csv('./working directory/오징어_페루_냉동지느러미.csv')

### 4-2-2. 오징어중국

In [ ]:
df_c_c = df_cala[df_cala.ctry1 == '중국']

In [ ]:
df_c_c.price.groupby(by = df_c_c.import_type).count()

import_type
건조                   5
냉동                 224
냉동,다리              101
냉동,다리,자숙             2
냉동,동체              221
냉동,동체,자숙            62
냉동,동체,포장횟감           2
냉동,슬라이스(S)          31
냉동,슬라이스(S),포장횟감      5
냉동,자숙                4
냉동,지느러미             54
Name: price, dtype: int64

In [ ]:
df_c_c[df_c_c.import_type == '냉동']
#'냉동', '냉동,동체'

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
1758,2016-03-07,중국,중국,외화획득용 원료,오징어,냉동,1.066601,2016
3521,2016-05-09,중국,중국,판매용,오징어,냉동,1.420320,2016
5051,2016-07-04,중국,중국,판매용,오징어,냉동,1.060000,2016
8820,2016-11-21,중국,중국,판매용,오징어,냉동,2.750000,2016
9170,2016-11-28,중국,중국,판매용,오징어,냉동,2.738958,2016
...,...,...,...,...,...,...,...,...
50783,2020-12-07,중국,중국,판매용,오징어,냉동,3.185604,2020
50969,2020-12-14,중국,중국,판매용,오징어,냉동,3.152568,2020
51033,2020-12-21,중국,중국,자사제품제조용,오징어,냉동,3.677745,2020
51141,2020-12-21,중국,중국,판매용,오징어,냉동,3.444083,2020


In [ ]:
df_c_c[df_c_c.import_type == '냉동,동체']

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
294,2016-01-11,중국,중국,판매용,오징어,"냉동,동체",1.149827,2016
497,2016-01-18,중국,중국,판매용,오징어,"냉동,동체",0.762367,2016
940,2016-02-01,중국,중국,판매용,오징어,"냉동,동체",2.481041,2016
1166,2016-02-15,중국,중국,판매용,오징어,"냉동,동체",2.968206,2016
1884,2016-03-14,중국,중국,판매용,오징어,"냉동,동체",0.665000,2016
...,...,...,...,...,...,...,...,...
50581,2020-11-30,중국,중국,판매용,오징어,"냉동,동체",1.571642,2020
50723,2020-12-07,중국,중국,판매용,오징어,"냉동,동체",2.614293,2020
50945,2020-12-14,중국,중국,판매용,오징어,"냉동,동체",1.539023,2020
51099,2020-12-21,중국,중국,판매용,오징어,"냉동,동체",1.763122,2020


In [ ]:
df_c_c_1 = df_c_c[df_c_c.import_type == '냉동,동체']
df_c_c_2 = df_c_c[df_c_c.import_type == '냉동']

In [ ]:
df_c_c_1 = df_c_c_1[df_c_c_1.purpose == '판매용']

In [ ]:
df_c_c_2 = df_c_c_2[df_c_c_2.purpose == '판매용']

In [ ]:
df_c_c_2.groupby(by = df_c_c_2.year).count()

,date,ctry1,ctry2,purpose,cat2,import_type,price
year,,,,,,,
2016,7,7,7,7,7,7,7
2017,40,40,40,40,40,40,40
2018,52,52,52,52,52,52,52
2019,52,52,52,52,52,52,52
2020,51,51,51,51,51,51,51


In [ ]:
df_c_c_1.groupby(by = df_c_c_1.year).count()

,date,ctry1,ctry2,purpose,cat2,import_type,price
year,,,,,,,
2016,30,30,30,30,30,30,30
2017,33,33,33,33,33,33,33
2018,44,44,44,44,44,44,44
2019,49,49,49,49,49,49,49
2020,47,47,47,47,47,47,47


In [ ]:
df_c_c_1 = cleaning(df_c_c_1)
df_c_c_2 = cleaning(df_c_c_2)

In [ ]:
df_c_c_1.to_csv('./working directory/오징어_중국_냉동동체.csv')
df_c_c_2.to_csv('./working directory/오징어_중국_냉동.csv')

### 4-2-3. 칠레

In [ ]:
df_c_v = df_cala[df_cala.ctry1 == '베트남']

In [ ]:
df_c_v.groupby(by = df_c_v.import_type).count()

,date,ctry1,ctry2,purpose,cat2,price,year
import_type,,,,,,,
건조,180,180,180,180,180,180,180
냉동,30,30,30,30,30,30,30
"냉동,다리",1,1,1,1,1,1,1
"냉동,다리,포장횟감",80,80,80,80,80,80,80
"냉동,동체",8,8,8,8,8,8,8
"냉동,동체,자숙,포장횟감",16,16,16,16,16,16,16
"냉동,동체,포장횟감",10,10,10,10,10,10,10
"냉동,슬라이스(S),포장횟감",163,163,163,163,163,163,163


In [ ]:
df_c_chile = df_cala[df_cala.ctry1 == '칠레']

In [ ]:
df_c_chile.groupby(by = df_c_chile.import_type).count()

,date,ctry1,ctry2,purpose,cat2,price,year
import_type,,,,,,,
냉동,11,11,11,11,11,11,11
"냉동,난포선",3,3,3,3,3,3,3
"냉동,다리",194,194,194,194,194,194,194
"냉동,다리,자숙",1,1,1,1,1,1,1
"냉동,동체",216,216,216,216,216,216,216
"냉동,동체,자숙",8,8,8,8,8,8,8
"냉동,지느러미",219,219,219,219,219,219,219


In [ ]:
df_chile1 = df_c_chile[df_c_chile.import_type == '냉동,동체']
df_chile2 = df_c_chile[df_c_chile.import_type == '냉동,다리']
df_chile3 = df_c_chile[df_c_chile.import_type == '냉동,지느러미']


In [ ]:
df_chile1 = df_chile1[df_chile1.purpose == '판매용']

In [ ]:
df_chile2 = df_chile2[df_chile2.purpose == '판매용']

In [ ]:
df_chile3 = df_chile3[df_chile3.purpose == '판매용']

In [ ]:
df_chile1 = cleaning(df_chile1)
df_chile2 = cleaning(df_chile2)
df_chile3 = cleaning(df_chile3)

In [ ]:
df_chile1 

,price
date,
2016-01-04,1.024569
2016-01-11,0.943622
2016-01-18,0.962642
2016-01-25,0.938800
2016-02-01,1.011010
...,...
2020-10-12,2.153563
2020-10-19,1.961280
2020-11-02,2.272553


In [ ]:
df_chile1.to_csv('./working directory/오징어_칠레_냉동동체.csv')
df_chile2.to_csv('./working directory/오징어_칠레_냉동다리.csv')
df_chile3.to_csv('./working directory/오징어_칠레_냉동지느러미.csv')

## 4-3. 새우

In [ ]:
df_shrimp = df[df.cat2 == '새우']

In [ ]:
df_shrimp
# 베트남, 태국, 중국

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
14,2016-01-04,중국,중국,판매용,새우,"냉동,살,자숙",8.204321,2016
24,2016-01-04,인도,인도,판매용,새우,냉동,9.059677,2016
25,2016-01-04,인도,인도,판매용,새우,냉동,1.934255,2016
33,2016-01-04,말레이시아,말레이시아,판매용,새우,냉동,7.189045,2016
34,2016-01-04,말레이시아,말레이시아,판매용,새우,냉동,7.812270,2016
...,...,...,...,...,...,...,...,...
51276,2020-12-28,태국,태국,판매용,새우,"냉동,살,자숙,포장횟감",21.040000,2020
51281,2020-12-28,페루,페루,판매용,새우,냉동,6.140000,2020
51284,2020-12-28,태국,태국,판매용,새우,"냉동,살",9.620000,2020
51306,2020-12-28,중국,중국,판매용,새우,건조,10.180000,2020


### 4-3-1. 새우 베트남

In [ ]:
df_shrimp_v = df_shrimp[df_shrimp.ctry1 == '베트남']

In [ ]:
df_shrimp_v.groupby(by = df_shrimp_v.import_type).count()

,date,ctry1,ctry2,purpose,cat2,price,year
import_type,,,,,,,
냉동,287,287,287,287,287,287,287
"냉동,살",422,422,422,422,422,422,422
"냉동,살,자숙",280,280,280,280,280,280,280
"냉동,살,자숙,포장횟감",223,223,223,223,223,223,223
"냉동,살,포장횟감",380,380,380,380,380,380,380
"냉동,자숙",52,52,52,52,52,52,52
염장,169,169,169,169,169,169,169


['냉동,살,포장횟감', '냉동,살', '냉동,살,자숙,포장횟감', '냉동', '냉동,살,자숙']

In [ ]:
v = df_shrimp_v

In [ ]:
v_1 = v[v.import_type == '냉동,살,포장횟감']
v_2 = v[v.import_type == '냉동,살']
v_3 = v[v.import_type == '냉동,살,자숙,포장횟감']
v_4 = v[v.import_type == '냉동']
v_5 = v[v.import_type == '냉동,살,자숙']


In [ ]:
v_1.groupby(by = v_1.date).count()

,ctry1,ctry2,purpose,cat2,import_type,price,year
date,,,,,,,
2016-01-04,2,2,2,2,2,2,2
2016-01-11,2,2,2,2,2,2,2
2016-01-18,2,2,2,2,2,2,2
2016-01-25,1,1,1,1,1,1,1
2016-02-01,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
2020-11-02,1,1,1,1,1,1,1
2020-11-23,1,1,1,1,1,1,1
2020-11-30,1,1,1,1,1,1,1


In [ ]:
v_1 = v_1[v_1.purpose == '판매용']

In [ ]:
v_1.groupby(by = v_1.date).count()

,ctry1,ctry2,purpose,cat2,import_type,price,year
date,,,,,,,
2016-01-04,2,2,2,2,2,2,2
2016-01-11,2,2,2,2,2,2,2
2016-01-18,2,2,2,2,2,2,2
2016-01-25,1,1,1,1,1,1,1
2016-02-01,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
2020-11-02,1,1,1,1,1,1,1
2020-11-23,1,1,1,1,1,1,1
2020-11-30,1,1,1,1,1,1,1


In [ ]:
l = v_1.price.groupby(by = v_1.date).mean()

In [ ]:
l

date
2016-01-04    11.551725
2016-01-11    13.336625
2016-01-18    11.754265
2016-01-25     9.667218
2016-02-01    10.246308
                ...    
2020-11-02    14.098655
2020-11-23    14.947158
2020-11-30    12.167809
2020-12-21    13.176159
2020-12-28    13.570000
Name: price, Length: 234, dtype: float64

In [ ]:
a = pd.DataFrame(l)

In [ ]:
a

,price
date,
2016-01-04,11.551725
2016-01-11,13.336625
2016-01-18,11.754265
2016-01-25,9.667218
2016-02-01,10.246308
...,...
2020-11-02,14.098655
2020-11-23,14.947158
2020-11-30,12.167809


In [ ]:
a.to_csv('./working directory/새우_베트남_냉동살포장횟감.csv')

In [ ]:
v_2

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
78,2016-01-04,베트남,베트남,판매용,새우,"냉동,살",8.691986,2016
83,2016-01-04,베트남,베트남,자사제품제조용,새우,"냉동,살",8.260303,2016
204,2016-01-04,베트남,베트남,판매용,새우,"냉동,살",7.562670,2016
283,2016-01-11,베트남,베트남,자사제품제조용,새우,"냉동,살",6.159457,2016
428,2016-01-11,베트남,베트남,판매용,새우,"냉동,살",8.123069,2016
...,...,...,...,...,...,...,...,...
50510,2020-11-30,베트남,베트남,판매용,새우,"냉동,살",10.737321,2020
50741,2020-12-07,베트남,베트남,판매용,새우,"냉동,살",7.746243,2020
50951,2020-12-14,베트남,베트남,판매용,새우,"냉동,살",7.593631,2020
51132,2020-12-21,베트남,베트남,판매용,새우,"냉동,살",7.989548,2020


In [ ]:
v_2 = v_2[v_2.purpose == '판매용']

In [ ]:
l2 = v_2.price.groupby(by = v_2.date).mean()

In [ ]:
b = pd.DataFrame(l2)

In [ ]:
b

,price
date,
2016-01-04,8.127328
2016-01-11,8.123069
2016-01-18,8.438049
2016-01-25,9.712139
2016-02-01,7.659386
...,...
2020-11-30,10.737321
2020-12-07,7.746243
2020-12-14,7.593631


In [ ]:
b.to_csv('./working directory/새우_베트남_냉동살.csv')

In [ ]:
v_3 = v_3[v_3.purpose == '판매용']

In [ ]:
v_3 = cleaning(v_3)

In [ ]:
v_3

,price
date,
2016-01-11,14.692197
2016-01-18,15.948179
2016-01-25,16.552750
2016-02-01,18.172198
2016-02-08,16.158020
...,...
2020-11-09,8.390197
2020-11-23,19.187066
2020-12-07,15.777495


In [ ]:
v_3.to_csv('./working directory/새우_베트남_냉동살자숙포장횟감.csv')

In [ ]:
v_4 = v_4[v_4.purpose == '판매용']

In [ ]:
v_4

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
77,2016-01-04,베트남,베트남,판매용,새우,냉동,10.083010,2016
157,2016-01-04,베트남,베트남,판매용,새우,냉동,7.453414,2016
353,2016-01-11,베트남,베트남,판매용,새우,냉동,7.162620,2016
576,2016-01-18,베트남,베트남,판매용,새우,냉동,6.576185,2016
591,2016-01-18,베트남,베트남,판매용,새우,냉동,10.105286,2016
...,...,...,...,...,...,...,...,...
47292,2020-08-03,베트남,베트남,판매용,새우,냉동,7.256762,2020
47525,2020-08-10,베트남,베트남,판매용,새우,냉동,7.017041,2020
48379,2020-09-14,베트남,베트남,판매용,새우,냉동,9.556716,2020
49098,2020-10-12,베트남,베트남,판매용,새우,냉동,7.359548,2020


In [ ]:
l4 = v_4.price.groupby(by = v_4.date).mean()

In [ ]:
c = pd.DataFrame(l4)

In [ ]:
c

,price
date,
2016-01-04,8.768212
2016-01-11,7.162620
2016-01-18,8.340736
2016-01-25,10.796673
2016-02-01,8.643117
...,...
2020-08-03,7.256762
2020-08-10,7.017041
2020-09-14,9.556716


In [ ]:
c.to_csv('./working directory/새우_베트남_냉동.csv')

In [ ]:
v_5 = v_5[v_5.purpose == '판매용']

In [ ]:
l5 = v_5.price.groupby(by = v_5.date).mean()

In [ ]:
d = pd.DataFrame(l5)

In [ ]:
d

,price
date,
2016-01-04,7.517132
2016-01-11,8.034134
2016-01-18,7.402905
2016-01-25,8.033705
2016-02-01,7.771798
...,...
2020-11-30,7.277901
2020-12-07,7.595661
2020-12-14,8.261538


In [ ]:
d.to_csv('./working directory/새우_베트남_냉동살자숙.csv')

### 4-3-2. 새우 태국

In [ ]:
thai = df_shrimp[df_shrimp.ctry1 == '태국']

In [ ]:
thai

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
106,2016-01-04,태국,태국,판매용,새우,"냉동,살,자숙",14.585459,2016
107,2016-01-04,태국,태국,판매용,새우,"냉동,살",16.355613,2016
144,2016-01-04,태국,태국,판매용,새우,냉장,9.377045,2016
155,2016-01-04,태국,태국,판매용,새우,냉장,9.021653,2016
260,2016-01-11,태국,태국,판매용,새우,냉동,7.776818,2016
...,...,...,...,...,...,...,...,...
51083,2020-12-21,태국,태국,판매용,새우,"냉동,살",10.962161,2020
51095,2020-12-21,태국,태국,판매용,새우,"냉동,살,자숙,포장횟감",17.438895,2020
51199,2020-12-28,태국,태국,판매용,새우,"냉동,살,자숙",15.780000,2020
51276,2020-12-28,태국,태국,판매용,새우,"냉동,살,자숙,포장횟감",21.040000,2020


In [ ]:
thai.groupby(by = thai.import_type).count()

,date,ctry1,ctry2,purpose,cat2,price,year
import_type,,,,,,,
냉동,24,24,24,24,24,24,24
"냉동,살",199,199,199,199,199,199,199
"냉동,살,자숙",147,147,147,147,147,147,147
"냉동,살,자숙,포장횟감",213,213,213,213,213,213,213
"냉동,살,포장횟감",1,1,1,1,1,1,1
"냉동,자숙",126,126,126,126,126,126,126
냉장,283,283,283,283,283,283,283


In [ ]:
## thai = ['냉장', '냉동,살,자숙,포장횟감']

thai_냉장 = thai[thai.import_type == '냉장']
thai_냉동살자숙포장 = thai[thai.import_type == '냉동,살,자숙,포장횟감']


In [ ]:
thai_냉장 = thai_냉장[thai_냉장.purpose == '판매용']

In [ ]:
thai_냉장

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
144,2016-01-04,태국,태국,판매용,새우,냉장,9.377045,2016
155,2016-01-04,태국,태국,판매용,새우,냉장,9.021653,2016
335,2016-01-11,태국,태국,판매용,새우,냉장,10.727038,2016
408,2016-01-11,태국,태국,판매용,새우,냉장,9.265868,2016
547,2016-01-18,태국,태국,판매용,새우,냉장,19.028442,2016
...,...,...,...,...,...,...,...,...
40966,2019-12-02,태국,태국,판매용,새우,냉장,11.500000,2019
41174,2019-12-09,태국,태국,판매용,새우,냉장,11.500000,2019
41405,2019-12-16,태국,태국,판매용,새우,냉장,11.500000,2019
41650,2019-12-23,태국,태국,판매용,새우,냉장,11.500000,2019


In [ ]:
thai_l1 = thai_냉장.price.groupby(by = thai_냉장.date).mean()

In [ ]:
thai_l1

date
2016-01-04     9.199349
2016-01-11     9.996453
2016-01-18    14.206107
2016-01-25    14.239936
2016-02-01    14.745228
                ...    
2019-12-02    11.500000
2019-12-09    11.500000
2019-12-16    11.500000
2019-12-23    11.500000
2019-12-30    11.500000
Name: price, Length: 181, dtype: float64

In [ ]:
x = pd.DataFrame(thai_l1)

In [ ]:
x

,price
date,
2016-01-04,9.199349
2016-01-11,9.996453
2016-01-18,14.206107
2016-01-25,14.239936
2016-02-01,14.745228
...,...
2019-12-02,11.500000
2019-12-09,11.500000
2019-12-16,11.500000


In [ ]:
x.to_csv('./working directory/새우_태국_냉장.csv')

In [ ]:
thai_냉동살자숙포장.purpose.unique()

array(['판매용'], dtype=object)

In [ ]:
thai_냉동살자숙포장 = cleaning(thai_냉동살자숙포장)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
thai_냉동살자숙포장

,price
date,
2016-01-11,20.415969
2016-01-25,18.780651
2016-02-15,19.946290
2016-02-29,20.450451
2016-03-14,19.037789
...,...
2020-11-23,16.108291
2020-12-07,13.932981
2020-12-14,16.991375


In [ ]:
thai_냉동살자숙포장.to_csv('./working directory/새우_태국_냉동살자숙포장.csv')

### 4-3-3.새우 중국

In [ ]:
china = df_shrimp[df_shrimp.ctry1 == '중국']

In [ ]:
china.groupby(by = china.import_type).count()

## ['건조', '냉동,살', '염장']

,date,ctry1,ctry2,purpose,cat2,price,year
import_type,,,,,,,
건조,228,228,228,228,228,228,228
냉동,16,16,16,16,16,16,16
"냉동,살",329,329,329,329,329,329,329
"냉동,살,자숙",24,24,24,24,24,24,24
염장,185,185,185,185,185,185,185


In [ ]:
china_건조 = china[china.import_type == '건조']
china_냉동살 = china[china.import_type == '냉동,살']
china_염장 = china[china.import_type == '염장']

In [ ]:
china_건조

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
136,2016-01-04,중국,중국,판매용,새우,건조,5.000000,2016
341,2016-01-11,중국,중국,판매용,새우,건조,5.242424,2016
538,2016-01-18,중국,중국,판매용,새우,건조,5.600589,2016
767,2016-01-25,중국,중국,판매용,새우,건조,4.834842,2016
993,2016-02-01,중국,중국,판매용,새우,건조,5.968992,2016
...,...,...,...,...,...,...,...,...
50515,2020-11-30,중국,중국,판매용,새우,건조,8.448052,2020
50745,2020-12-07,중국,중국,판매용,새우,건조,8.907333,2020
50938,2020-12-14,중국,중국,판매용,새우,건조,9.711301,2020
51097,2020-12-21,중국,중국,판매용,새우,건조,9.968652,2020


In [ ]:
len(china_건조.date.unique())

228

In [ ]:
china_건조 = cleaning(china_건조)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
china_건조

,price
date,
2016-01-04,5.000000
2016-01-11,5.242424
2016-01-18,5.600589
2016-01-25,4.834842
2016-02-01,5.968992
...,...
2020-11-30,8.448052
2020-12-07,8.907333
2020-12-14,9.711301


In [ ]:
china_건조.to_csv('./working directory/새우_중국_건조.csv')


#china_건조.to_csv(r'/content/drive/MyDrive/Colab Notebooks/bigcon/files/df_shrimp_china_dried.csv')

In [ ]:
len(china_냉동살.date.unique())

212

In [ ]:
china_냉동살

,date,ctry1,ctry2,purpose,cat2,import_type,price,year
137,2016-01-04,중국,중국,판매용,새우,"냉동,살",6.331636,2016
369,2016-01-11,중국,중국,판매용,새우,"냉동,살",7.077197,2016
388,2016-01-11,중국,중국,판매용,새우,"냉동,살",5.594349,2016
446,2016-01-18,중국,중국,판매용,새우,"냉동,살",6.940614,2016
521,2016-01-18,중국,중국,판매용,새우,"냉동,살",6.000405,2016
...,...,...,...,...,...,...,...,...
47392,2020-08-10,중국,중국,판매용,새우,"냉동,살",7.306287,2020
47773,2020-08-24,중국,중국,판매용,새우,"냉동,살",6.435755,2020
49044,2020-10-12,중국,중국,판매용,새우,"냉동,살",8.009503,2020
49250,2020-10-19,중국,중국,판매용,새우,"냉동,살",6.243344,2020


In [ ]:
l_china = china_냉동살.price.groupby(by = china_냉동살.date).mean()

In [ ]:
y = pd.DataFrame(l_china)

In [ ]:
y

,price
date,
2016-01-04,6.331636
2016-01-11,6.335773
2016-01-18,6.470509
2016-01-25,6.770484
2016-02-01,6.569193
...,...
2020-08-10,7.306287
2020-08-24,6.435755
2020-10-12,8.009503


In [ ]:
y.to_csv('./working directory/새우_중국_냉동살.csv')

In [ ]:
len(china_염장.date.unique())

175

In [ ]:
l_china2 = china_염장.price.groupby(by = china_염장.date).mean()

In [ ]:
z = pd.DataFrame(l_china2)

In [ ]:
z.to_csv('./working directory/새우_중국_염장.csv')

In [ ]:
새우_베트남_냉동살자숙포장횟감 = database['새우_베트남_냉동살자숙포장횟감']
새우_베트남_냉동살자숙 = database['새우_베트남_냉동살자숙']
새우_베트남_냉동살 = database['새우_베트남_냉동살']
새우_베트남_냉동 = database['새우_베트남_냉동']
새우_중국_건조 = database['새우_중국_건조']
새우_중국_냉동살 = database['새우_중국_냉동살']
새우_중국_염장 = database['새우_중국_염장']
새우_태국_냉동살자숙포장횟감 = database['새우_태국_냉동살자숙포장횟감']
새우_태국_냉장 = database['새우_태국_냉장']
연어_노르웨이_냉장_자사제품제조용 = database['연어_노르웨이_냉장_자사제품제조용']
연어_노르웨이_냉장_판매용 = database['연어_노르웨이_냉장_판매용']
연어_노르웨이_냉장포장횟감필렛 = database['연어_노르웨이_냉장포장횟감필렛']
연어_노르웨이_냉장필렛_자사제품제조용 = database['연어_노르웨이_냉장필렛_자사제품제조용']
연어_노르웨이_냉장필렛_판매용 = database['연어_노르웨이_냉장필렛_판매용']
연어_노르웨이_냉장필렛횟감 = database['연어_노르웨이_냉장필렛횟감']
오징어_중국_냉동동체 = database['오징어_중국_냉동동체']
오징어_중국_냉동 = database['오징어_중국_냉동']
오징어_칠레_냉동다리 = database['오징어_칠레_냉동다리']
오징어_칠레_냉동동체 = database['오징어_칠레_냉동동체']
오징어_칠레_냉동지느러미 = database['오징어_칠레_냉동지느러미']
오징어_페루_냉동다리 = database['오징어_페루_냉동다리']
오징어_페루_냉동동체자숙_자사제품제조용 = database['오징어_페루_냉동동체자숙_자사제품제조용']
오징어_페루_냉동동체자숙_판매용 = database['오징어_페루_냉동동체자숙_판매용']
오징어_페루_냉동지느러미 = database['오징어_페루_냉동지느러미']



